
# Scalar, Vector, and Tensor Fields in gwexpy

This tutorial demonstrates the usage of `ScalarField`, `VectorField`, and `TensorField` classes for representing physical fields in 4D space-time (or other domains).

These classes are designed to:
1.  **Preserve Physics**: Maintain units and axis metadata through operations.
2.  **Support Domains**: Explicitly handle Time/Frequency and Real/k-space domains.
3.  **Enable Batch Processing**: Perform signal processing (FFT, filtering) on all vector/tensor components simultaneously.
    

In [1]:

import numpy as np
from astropy import units as u

from gwexpy.fields import TensorField, VectorField, make_demo_scalar_field

# For reproducibility
np.random.seed(42)



## 1. ScalarField

`ScalarField` is the fundamental building block. It wraps a 4D array `(axis0, x, y, z)` with rich metadata.
    

In [2]:

# Create a DEMO scalar field (e.g., Electric Potential V)
# 1 second duration, 100 Hz, 4x4x4 spatial grid
# nt=100, dt=0.01s
f_scalar = make_demo_scalar_field(
    nt=100,
    nx=4, ny=4, nz=4,
    dt=0.01 * u.s,
    dx=0.1 * u.m,
    unit=u.V
)

print(f"Shape: {f_scalar.shape}")
print(f"Unit: {f_scalar.unit}")
print(f"Axis 0 (Time) domain: {f_scalar.axis0_domain}")


Shape: (100, 4, 4, 4)
Unit: V
Axis 0 (Time) domain: time



### Slicing & Physics Preservation
Slicing a `ScalarField` always returns a `ScalarField`. Even if you select a single point, the 4D structure is preserved (size 1 axes).
    

In [3]:

# Slice at x=0, y=1, z=2
f_slice = f_scalar.isel(x=0, y=1, z=2)
print(f"Sliced shape: {f_slice.shape}")

# FFT along time axis (returns freq domain ScalarField)
f_freq = f_slice.fft_time()
print(f"FFT domain: {f_freq.axis0_domain}")
print(f"FFT unit: {f_freq.unit}")


Sliced shape: (100, 1, 1, 1)
FFT domain: frequency
FFT unit: V



## 2. VectorField

`VectorField` is a collection of `ScalarField` components (e.g., 'x', 'y', 'z'). It supports geometric vector algebra.
    

In [4]:

# Construct a VectorField from components
# Example: Electric Field E = (Ex, Ey, Ez)
# We use copy() and in-place operations to preserve metadata
Ex = f_scalar.copy()
Ey = f_scalar.copy()
Ey *= 0.5
Ez = f_scalar.copy()
Ez *= 0.0

E_field = VectorField({'x': Ex, 'y': Ey, 'z': Ez})
print(f"Components: {list(E_field.keys())}")


Components: ['x', 'y', 'z']



### Geometric Operations
Common vector operations like Dot product, Cross product, and Norm are supported and return `ScalarField` or `VectorField` with correct units.
    

In [5]:

# 1. Magnitude (Norm) -> ScalarField
E_mag = E_field.norm()
print(f"Magnitude unit: {E_mag.unit}")
print(f"Max Magnitude: {E_mag.value.max():.2f}")

# 2. Dot Product (Self dot should equal mag^2) -> ScalarField
E_dot_E = E_field.dot(E_field)
print(f"Dot product unit: {E_dot_E.unit}")

# 3. Cross Product -> VectorField
# Let's define a B-field aligned with Z
Bx = f_scalar.copy()
Bx *= 0.0
By = f_scalar.copy()
By *= 0.0
Bz = f_scalar.copy()
Bz *= 1.0 # 1 Tesla

B_field = VectorField({'x': Bx, 'y': By, 'z': Bz})

# Poynting Vector-like S = E x B
S_vec = E_field.cross(B_field)
print(f"Cross product components: {list(S_vec.keys())}")
# E=(1, 0.5, 0), B=(0, 0, 1) => E x B = (0.5*1 - 0, 0 - 1*1, 0) = (0.5, -1, 0)
print(f"S_x max approx: {S_vec['x'].value.max():.2f}")


Magnitude unit: V
Max Magnitude: 1.01
Dot product unit: V2
Cross product components: ['x', 'y', 'z']
S_x max approx: 0.41



### Batch Signal Processing
You can apply signal processing (FFT, Filter, Resample) to all components at once.
    

In [6]:

# FFT all components
E_freq = E_field.fft_time_all()
print(f"E_freq['x'] domain: {E_freq['x'].axis0_domain}")

# Filter all components (Lowpass 10Hz)
from gwpy.signal import filter_design

lp = filter_design.lowpass(10.0, 100.0) # 10Hz cutoff, 100Hz sampling
E_filtered = E_field.filter_all(lp)


E_freq['x'] domain: frequency



## 3. TensorField

`TensorField` represents rank-2 (matrix) fields, such as Stress or Strain tensors. Components are indexed by tuples `(i, j)`.
    

In [7]:

# Create a 2x2 TensorField (Rank 2)
# T = [[2, 1], [0, 3]] * f_scalar
f00 = f_scalar.copy(); f00 *= 2
f01 = f_scalar.copy(); f01 *= 1
f10 = f_scalar.copy(); f10 *= 0
f11 = f_scalar.copy(); f11 *= 3

T = TensorField({
    (0, 0): f00, (0, 1): f01,
    (1, 0): f10, (1, 1): f11
}, rank=2)

# Trace -> ScalarField (sum of diagonals: 2+3=5)
tr = T.trace()
print(f"Trace max value (approx 5 * max(f)): {tr.value.max():.2f}")

# Determinant -> ScalarField (ad - bc: 6 - 0 = 6)
det = T.det()
print(f"Det max value (approx 6 * max(f)^2): {det.value.max():.2f}")


Trace max value (approx 5 * max(f)): 4.50
Det max value (approx 6 * max(f)^2): 4.87



### Matrix-Vector Multiplication
We can multiply a TensorField with a VectorField using the `@` operator.
    

In [8]:

# Define a 2D vector v = (x, y)
vy = f_scalar.copy(); vy *= 2
v_2d = VectorField({'x': f_scalar, 'y': vy}) # (1, 2)

# T @ v = [[2, 1], [0, 3]] @ [1, 2] = [2*1 + 1*2, 0*1 + 3*2] = [4, 6]
# Note: TensorField automatically maps 0->x, 1->y
result_vec = T @ v_2d

print(f"Result components: {list(result_vec.keys())}")
# Check scaling
# Input f_scalar max is M. result x should be approx 4*M
print(f"Result X max ratio: {result_vec['x'].value.max() / f_scalar.value.max():.2f}")


Result components: ['x', 'y']
Result X max ratio: 3.60
